In [ ]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time

# Crear archivo y Titulos del CSV
filename = "departamentosEstreno.csv"
f= open(filename, "w")
headers ="Dirección, Precio, Nro de Departamentos, Nro de Dormitorios, Área, Baños, Estacionamientos, Estado, Entrega, Servicios, Fecha de Publicación, Link\n"
f.write(headers)

# Realizar la busqueda de departamentos
for pagina in range(1,47):
    print("Pagina Numero ", pagina)
    browser = webdriver.Chrome("C:\chromedriver.exe")
    if (pagina == 1):
        paginaBusqueda = "https://www.adondevivir.com/departamentos-en-venta-en-santiago-de-surco-a-estrenar.html"
#         paginaBusqueda = "https://www.adondevivir.com/departamentos-en-venta-en-santiago-de-surco.html#"
    else:
        paginaBusqueda = "https://www.adondevivir.com/departamentos-en-venta-en-santiago-de-surco-a-estrenar-pagina-" + str(pagina) + ".html"
#         paginaBusqueda = "https://www.adondevivir.com/departamentos-en-venta-en-santiago-de-surco-pagina-" + str(pagina) + ".html"
    
    browser.get(paginaBusqueda)
    page_html = browser.page_source
    page_soup = soup(page_html, "html.parser")
    containers = page_soup.findAll("div", {"class":"postingCardInfo"})

    # Iterar sobre cada resultado de la busqueda 
    for container in containers:

        # Obtener Direccion
        direccionSpan = container.findAll("span", {"class":"postingCardLocationTitle"})
        direccion = direccionSpan[0].text.strip('\n\r\tS/": ')
        print(direccion)
        # Obtener Precio
    #     precioSpan = container.findAll("span", {"class":"firstPrice"})
    #     precio = precioSpan[0].text.strip('\n\r\tS/": ')
    #     print(precio)

        # Obtener nro de Departamentos, Area, nro de Dormitorios, nro de Baños, nro de Estacionamientos
        data = container.findAll("li")

        departamentos = 'None'
        dormitorios = 'None'
        area = 'None'
        baños = "None"
        estacionamientos = "None"

        for i in range(len(data)):
            line = data[i].findAll("i")
            if "iconUnits" in line[0]['class']:
                departamentos = data[i].text.strip('tuni.\n\r\t": ')
            elif "iconBedrooms" in line[0]['class']:
                dormitorios = data[i].text.strip('dorm.\n\r\t": ')
            elif "iconArea" in line[0]['class']:
                area = data[i].text.strip('m².\n\r\t": ')
            elif "iconBathroom" in line[0]['class']:
                baños = data[i].text
            elif "iconGarage" in line[0]['class']:
                estacionamientos = data[i].text.replace("estac.", "").strip()

        # Ir al enlace
        linkList = container.findAll("a", {"class":"go-to-posting"})
        newLink = "https://www.adondevivir.com" + linkList[0]['href'] 
        print(newLink)
        browser = webdriver.Chrome("C:\chromedriver.exe")
        browser.get(newLink)
        time.sleep(1)
        page_html = browser.page_source

        # Conseguir datos adicionales
        page_soup2 = soup(page_html, "html.parser")
        containers2 = page_soup2.findAll("div", {"id":"reactGeneralFeatures"})
        if (len(containers2) > 0):
            container2 = containers2[0]
            info = container2.findAll("li")
            if (info):
                servicios = info[0].text.strip()
            else:
                servicios = "None"
            if (info):
                for i in range(1, len(info)):
                    servicios = servicios + '-' + info[i].text.strip()

        # Conseguir estado de la obra

        containers4 = page_soup2.findAll("div", {"class":"current"})
        if (len(containers4) > 0 ):
            estado = containers4[0].text.strip('\n\r\tS/",: ')
        else:
            estado = "None"

        # Conseguir fecha de entrega
        containers5 = page_soup2.findAll("div", {"class":"row"})
        if (len(containers5) > 0):    
            entrega = containers5[0].text.strip('\n\r\t",: ').replace("\n\t\t\t\t\n\n", " ")
        else:
            entrega = "None"

        # Conseguir nro de estacionamientos
        if (estacionamientos == "None"):
            if (len(containers5)> 5):
                estacionamientos = containers5[5].text.strip('\n\r\t",: ').replace("\n\t\t\t\t\n\n", " ")

        # Conseguir fecha de publicacion

        containers6 = page_soup2.findAll("span", {"class":"section-date"})
        if (len(containers6) > 0):
            fechaPub = containers6[0].text.replace("Publicado hace", "").strip('\n\r\tS/",: ')
        else:
            fechaPub = "None"

        # Conseguir datos de cada departamento

        containers3 = page_soup2.findAll("div", {"class": "AccordionItem-td4pyq-0"} )
        if (len(containers3) > 0):
            for i in range(len(containers3)):
                dormitorios = containers3[i].findAll("h4", {"class": "AccordionTitle-td4pyq-2"})[0].text
                dormitorios = dormitorios[17:18]
                data3 = containers3[i].findAll("div", {"class": "sc-bkzZxe"})
                for j in range(len(data3)):
                    precioText = data3[j].findAll("div", {"class": "sc-idOhPF" })
                    areaText = data3[j].findAll("li")
                    precio = precioText[0].text.strip('\n\r\tS/",: ').replace(",", "")
                    area = areaText[0].text
                    if (len(areaText) > 1):
                        baños = areaText[1].text.strip('\n\r\tS/"bBaños,: ')
                    else:
                        baños = "None"
                    print("Direccion", direccion, "; Precio", precio, "; Departamentos", departamentos, 
                          "; Dormitorios", dormitorios, "; Area", area, "; Baños", baños, "; Estacionamientos", estacionamientos)

                    f.write(direccion.replace(",", "-") + "," + precio + "," + departamentos + "," + dormitorios + "," + area + "," 
                    + baños + "," + estacionamientos.replace("Estacionamientos ", "") + "," + estado + "," + 
                    entrega.replace("Entrega ", "") + "," + servicios + "," + fechaPub + "," + newLink + "\n") 
                
f.close()